This notebook process a text file (specified in `INPUT_FILE`), which should be of the form one search string per line. 

It attempts to automatically match each line in the file to a SNOMED code. 
If this is not possible, it will prompt the user to enter a relevant code. 

Mappings are saved incrementally to `.json` files

# Setup

## Imports

In [1]:

from condition_mapping import ConditionMapper
from snomed import Snomed

snomed = Snomed()


## Function Definitions

# Configuration

In [2]:
# INPUT_FILE = 'data/inputs_example.txt' # Contains conditions that directly map to SNOMED-CT
# INPUT_FILE = 'data/inputs_example_partial.txt' # Contains a condition ('sneez') that does not directly map to SNOMED-CT, but should be a partial match
# INPUT_FILE = 'data/inputs_example_unmatchable.txt' # Contains a condition ('abcdefghijklmnopqrstuvwxyz') for which no partial match should be found
INPUT_FILE = 'data/inputs_example_amalgamation.txt' 

CM = ConditionMapper(INPUT_FILE)
CM.save_mapping()

Saving mapping to data/inputs_example_amalgamation_mapped_5.json... Done


# Load Data and Automatic Mapping

In [3]:
CM.automatically_map_conditions(snomed)
CM.save_mapping()

Automatically mapping conditions to SNOMED-CT...
	Bronchiolitis mapped to Bronchiolitis (disorder)
	Pneumonia mapped to Pneumonia (disorder)
	Lower Respiratory Tract Infection mapped to Lower respiratory tract infection (disorder)
	Status epilepticus mapped to Status epilepticus (finding)
	Dravet Syndrome mapped to Severe myoclonic epilepsy in infancy (disorder)
	Lennox-Gastaut Syndrome mapped to Lennox-Gastaut syndrome (disorder)
	Myoclonic encephalopathy mapped to Myoclonic encephalopathy (disorder)
	Acute Bronchiolitis mapped to Acute bronchiolitis (disorder)
	Respiratory syncytial virus bronchiolitis mapped to Respiratory syncytial virus bronchiolitis (disorder)
9 conditions mapped to SNOMED-CT.
2 conditions not mapped to SNOMED-CT:
	chest infection
	Infantile Spasms
Saving mapping to data/inputs_example_amalgamation_mapped_6.json... Done


# Manual Mapping of CUIs

In [6]:
# This cell runs a loop, asking for manual input of CUIs for unmapped conditions. 
# To skip a condition, leave the CUI emtpy and press Enter.
#
# To stop the loop, interrupt the kernel (e.g. press the stop button in Jupyter Notebook), then leave the CUI empty and press Enter.
# Any matched CUIs previously entered will still be available in MANUALLY_MAPPED_CUIS.

CM.get_user_input_cuis(snomed)


'Manually mapped 0 conditions to SNOMED-CT:'

0 conditions skipped:


# Load - continue from previous session

In [ ]:
KNOWN_CONDITIONS, UNKNOWN_CONDITIONS = load_mapping_file(1)

In [ ]:
KNOWN_CONDITIONS, UNKNOWN_CONDITIONS = load_most_recent_mapping_file()

In [18]:
# Use with caution... this will delete all mapping files!
CM.clear_mapping_files()

In [ ]:
a = input('Press Enter to exit')
a